In [1]:
# takes 1h per 100k hands.
# Read acbl_hand_records.pkl file and clean up double dummy (DD) and par errors.
# Best to re-use previous cleaned up results as performance is slow. Use previous results unless record is new.
# It's not sufficient to validate par score vul calcs by simply correcting vul because some scores would be missing.
# Creates ../acbl/acbl_hand_records_cleaned.pkl

# todo:
# acbl_hand_records.pkl are incorrect due to legacy ignoring of vul. Clean here or correct sql in previous step? 
# figure out why this line throws an exception: stdhrs.at[grp_start+handno,'Par'] = pars
# board_record_string has replaced '10' with 'T'. This is confusing as it doesn't match sql. Needed?

# next steps:
# merge-hand-records.ipynb merges acbl and tcg standardized hand records?

# previous steps:
# acbl_sql_to_hand_records.ipynb creates acbl_hand_records.pkl

# Requirements
# https://github.com/Afwas/python-dds with some of my mods.

In [2]:
import pandas as pd
import pathlib
from IPython.display import display # needed to define display() method in vscode
import sys

In [3]:
sys.path.append(str(pathlib.Path.cwd().parent.joinpath('mlBridgeLib')))
sys.path
import mlBridgeLib

In [4]:
# override pandas display options
mlBridgeLib.pd_options_display()

In [5]:
rootPath = pathlib.Path('e:/bridge/data')
acblPath = rootPath.joinpath('acbl')

In [6]:
# takes 20s
acbl_hand_records_filename = 'acbl_hand_records.pkl'
acbl_hand_records_file = acblPath.joinpath(acbl_hand_records_filename)
stdhrs = pd.read_pickle(acbl_hand_records_file)
display(len(stdhrs),stdhrs)

4012439

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,DDmakes
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))","(100, [(4, S, *, EW, -1)])","((10, 4, 10, 4, 8), (2, 9, 3, 9, 2), (10, 4, 10, 4, 8), (2, 9, 3, 9, 2))"
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))","(90, [(1, D, , NS, 1), (1, C, , NS, 1), (1, N, , NS, 0)])","((8, 8, 7, 7, 7), (4, 5, 6, 6, 6), (8, 8, 7, 7, 7), (4, 5, 6, 6, 6))"
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJTD65CAKTSK984H5DT8743C987SA52HAQ9832DKJC42,"((T, 764, AQ92, QJ653), (K984, 5, T8743, 987), (A52, AQ9832, KJ, 42), (QJ763, KJT, 65, AKT))","(100, [(3, S, *, EW, -1)])","((9, 7, 9, 4, 6), (4, 6, 3, 8, 6), (9, 7, 9, 5, 6), (4, 6, 3, 8, 6))"
3,"(acbl, 2620420)",77908,100961,28,200217A_28,W,N_S,STHQJ9632DJ9CJ932SKQH8DQT86542CT76SJ76HATDAK3CAKQ84SA985432HK754D7C5,"((T, QJ9632, J9, J932), (J76, AT, AK3, AKQ84), (A985432, K754, 7, 5), (KQ, 8, QT86542, T76))","(-300, [(6, H, *, NS, -2)])","((1, 0, 10, 8, 1), (11, 11, 3, 5, 11), (0, 0, 10, 8, 0), (11, 11, 3, 5, 11))"
4,"(acbl, 2620399)",77908,100961,07,200217A_07,S,Both,S2HQT762D92CAKT92SAKJ7HA8DKJ4C8643SQT854HK43DA765CQS963HJ95DQT83CJ75,"((2, QT762, 92, AKT92), (QT854, K43, A765, Q), (963, J95, QT83, J75), (AKJ7, A8, KJ4, 8643))","(-1430, [(6, S, , EW, 0)])","((6, 2, 6, 1, 3), (7, 11, 6, 12, 8), (6, 2, 6, 1, 3), (7, 11, 6, 12, 8))"
5,"(acbl, 2620396)",77908,100961,04,200217A_04,W,Both,S84HAKJ83D4CQT743SA65HQ62DQ8732CK8SJ972HT5DAKJ96C95SKQT3H974DT5CAJ62,"((84, AKJ83, 4, QT743), (J972, T5, AKJ96, 95), (KQT3, 974, T5, AJ62), (A65, Q62, Q8732, K8))","(620, [(4, H, , NS, 0)])","((10, 5, 10, 6, 6), (3, 8, 3, 6, 6), (10, 5, 10, 6, 6), (3, 8, 3, 6, 6))"
6,"(acbl, 2620410)",77908,100961,18,200217A_18,E,N_S,S872HJ8753DQ873CASAJ65HQ4DAKJ4C874S93HAK6D962CQT963SKQT4HT92DT5CKJ52,"((872, J8753, Q873, A), (93, AK6, 962, QT963), (KQT4, T92, T5, KJ52), (AJ65, Q4, AKJ4, 874))","(-120, [(1, N, , EW, 1)])","((4, 4, 5, 5, 5), (9, 8, 7, 7, 8), (4, 5, 5, 5, 5), (9, 8, 7, 7, 8))"
7,"(acbl, 2620400)",77908,100961,08,200217A_08,W,None,S953H98642DJ9CKJ5SAJT6HAJ3DT2CAT96S742HKQ7DK764CQ43SKQ8HT5DAQ853C872,"((953, 98642, J9, KJ5), (742, KQ7, K764, Q43), (KQ8, T5, AQ853, 872), (AJT6, AJ3, T2, AT96))","(-140, [(1, S, , E, 2)])","((4, 6, 6, 4, 5), (8, 7, 7, 9, 8), (4, 6, 6, 4, 5), (8, 7, 7, 8, 8))"
8,"(acbl, 2620397)",77908,100961,05,200217A_05,N,N_S,S98HAKT83D84CAQ82SQJHQJ65DT2CJ7653SAK752H72DAK96CKTST643H94DQJ753C94,"((98, AKT83, 84, AQ82), (AK752, 72, AK96, KT), (T643, 94, QJ753, 94), (QJ, QJ65, T2, J7653))","(-120, [(2, N, , EW, 0)])","((5, 6, 7, 4, 5), (7, 7, 6, 8, 8), (5, 6, 7, 4, 5), (7, 7, 6, 8, 8))"
9,"(acbl, 2620398)",77908,100961,06,200217A_06,E,E_W,SA53HJT8DQ8743CQ9SKQJT87H954DJCA62S94HAQ62DAT965C54S62HK73DK2CKJT873,"((A53, JT8, Q8743, Q9), (94, AQ62, AT965, 54), (62, K73, K2, KJT873), (KQJT87, 954, J, A62))","(-140, [(2, S, , EW, 1)])","((7, 6, 5, 4, 5), (5, 7, 8, 9, 5), (7, 6, 5, 4, 5), (5, 7, 8, 9, 5))"


In [7]:
# todo: what to do with duplicate board_record_string? keep, discard? Keep dups to sustain integrity with board results?
print("unique brs count:",len(stdhrs['board_record_string'].unique()))
print("keep first duplicate brs count:",sum(stdhrs['board_record_string'].duplicated(keep='first')))
print("keep last duplicate brs count:",sum(stdhrs['board_record_string'].duplicated(keep='last')))
print("keep False duplicate brs count:",sum(stdhrs['board_record_string'].duplicated(keep=False)))
stdhrs.sort_values('board_record_string')

unique brs count: 1949959
keep first duplicate brs count: 2062480
keep last duplicate brs count: 2062480
keep False duplicate brs count: 2359048


,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,DDmakes
2600422,"(acbl, 562562)",16921,19796,20,191010A_20,W,Both,S2H2D543CAKQJT952SAQJ7HAKQT874D8C3SK98H653DKJ76C876ST6543HJ9DAQT92C4,"((2, 2, 543, AKQJT952), (K98, 653, KJ76, 876), (T6543, J9, AQT92, 4), (AQJ7, AKQT874, 8, 3))","(-100, [(6, C, *, NS, -1)])","((11, 9, 2, 4, 2), (2, 3, 11, 9, 3), (11, 9, 2, 4, 2), (2, 3, 11, 8, 2))"
2600386,"(acbl, 562526)",16920,19795,20,191010A_20,W,Both,S2H2D543CAKQJT952SAQJ7HAKQT874D8C3SK98H653DKJ76C876ST6543HJ9DAQT92C4,"((2, 2, 543, AKQJT952), (K98, 653, KJ76, 876), (T6543, J9, AQT92, 4), (AQJ7, AKQT874, 8, 3))","(-100, [(6, C, *, NS, -1)])","((11, 9, 2, 4, 2), (2, 3, 11, 9, 3), (11, 9, 2, 4, 2), (2, 3, 11, 8, 2))"
1918190,"(acbl, 448172)",13494,15657,11,190928A_11,S,None,S2H2D87542CKJ8543SJT974HQJ97DATCQ6SA83HAT6DKQ963CATSKQ65HK8543DJC972,"((2, 2, 87542, KJ8543), (A83, AT6, KQ963, AT), (KQ65, K8543, J, 972), (JT974, QJ97, AT, Q6))","(-430, [(3, N, , W, 1)])","((6, 3, 4, 3, 3), (6, 9, 8, 9, 8), (6, 3, 4, 3, 3), (5, 9, 8, 9, 10))"
3903056,"(acbl, 7994706)",315059,431317,22,211115A_22,E,E_W,S2H2D8754CAJT9543SA9853HAQJ943DQC2SKQHKT86DT9632C76SJT764H75DAKJCKQ8,"((2, 2, 8754, AJT9543), (KQ, KT86, T9632, 76), (JT764, 75, AKJ, KQ8), (A9853, AQJ943, Q, 2))","(-100, [(6, C, *, NS, -1)])","((11, 6, 2, 4, 4), (2, 6, 11, 8, 3), (11, 6, 2, 4, 4), (2, 6, 11, 8, 3))"
219045,"(acbl, 601942)",18107,21157,28,191014A_28,W,N_S,S2H2D876532CQ9853SAT943HQ53DQT94CJS875HAKT974DAKCKTSKQJ6HJ86DJCA7642,"((2, 2, 876532, Q9853), (875, AKT974, AK, KT), (KQJ6, J86, J, A7642), (AT943, Q53, QT94, J))","(-460, [(5, N, , EW, 0)])","((9, 6, 1, 3, 1), (4, 7, 10, 10, 11), (9, 6, 1, 3, 1), (4, 7, 10, 10, 11))"
3096995,"(acbl, 2335944)",69510,88292,32,200203A_32,W,E_W,S2H2D876543CT7432SAT53HKT64DAKJ9CKS864HJ87DQ2CQJ965SKQJ97HAQ953DTCA8,"((2, 2, 876543, T7432), (864, J87, Q2, QJ965), (KQJ97, AQ953, T, A8), (AT53, KT64, AKJ9, K))","(-120, [(2, N, , EW, 0)])","((4, 6, 5, 6, 4), (7, 6, 7, 7, 8), (4, 7, 6, 6, 5), (7, 6, 7, 7, 8))"
3181363,"(acbl, 1208228)",36150,44726,03,191121A_03,S,E_W,S2H2DAJ9654CA8765SAQJT95HAQT953D7CSK874HKJ874DTCT92S63H6DKQ832CKQJ43,"((2, 2, AJ9654, A8765), (K874, KJ874, T, T92), (63, 6, KQ832, KQJ43), (AQJT95, AQT953, 7, ))","(-300, [(7, C, *, NS, -2)])","((11, 10, 0, 0, 1), (1, 2, 12, 12, 2), (11, 10, 0, 0, 1), (1, 2, 12, 12, 2))"
1278408,"(acbl, 6461178)",254594,354737,07,210715A_07,S,Both,S2H2DAKJ9652CA954SJT84HT87DQT73CQ7SQ97HAKJ543DCKJ86SAK653HQ96D84CT32,"((2, 2, AKJ9652, A954), (Q97, AKJ543, , KJ86), (AK653, Q96, 84, T32), (JT84, T87, QT73, Q7))","(120, [(2, N, , NS, 0)])","((8, 9, 4, 7, 8), (4, 3, 8, 5, 3), (8, 9, 4, 7, 8), (4, 3, 8, 6, 4))"
2485077,"(acbl, 6727843)",265244,368064,35,210811A_35,S,E_W,S2H2DAKJ9753CJ542SKJ9HKQJT8DQTCAKTSAT7654HA543D6C73SQ83H976D842CQ986,"((2, 2, AKJ9753, J542), (AT7654, A543, 6, 73), (Q83, 976, 842, Q986), (KJ9, KQJT8, QT, AKT))","(-1100, [(7, D, *, NS, -5)])","((5, 8, 0, 0, 0), (7, 4, 12, 12, 6), (5, 8, 0, 0, 0), (7, 4, 12, 12, 6))"
3665157,"(acbl, 772707)",23224,27683,20,191028A_20,W,Both,S2H2DAKQ3CQJ75432SJ43HAT953DT974C8SKT976HQ6DJ8652C9SAQ85HKJ874DCAKT6,"((2, 2, AKQ3, QJ75432), (KT976, Q6, J8652, 9), (AQ85, KJ874, , AKT6), (J43, AT953, T974, 8))","(990, [(6, N, , NS, 0)])","((12, 8, 9, 8, 12), (0, 5, 3, 5, 1), (12, 8, 9, 8, 12), (0, 5, 3, 5, 1))"


In [8]:
assert len(stdhrs['board_record_string'].unique())+sum(stdhrs['board_record_string'].duplicated(keep='first'))==len(stdhrs)

In [9]:
# mark duplicate board_record_string.
stdhrs['brs_dup'] = stdhrs.duplicated(subset='board_record_string',keep=False)
sum(stdhrs['brs_dup'])

2359048

In [10]:
# experiment with drop_duplicates()
dups = stdhrs['board_record_string'].drop_duplicates()
len(stdhrs),len(dups),len(stdhrs)-len(dups)

(4012439, 1949959, 2062480)

In [11]:
# takes 20s + 1m + 1m
# similar to acbl_hand_records_dd_experiments
acbl_hand_records_cleaned_filename = 'acbl_hand_records_cleaned.pkl'
acbl_hand_records_cleaned_file = acblPath.joinpath(acbl_hand_records_cleaned_filename)
if acbl_hand_records_cleaned_file.exists():
    hrcdf = pd.read_pickle(acbl_hand_records_cleaned_file)
    display(len(hrcdf),hrcdf.head())
    # Preferred to use pd.merge but caused "out of memory" condition. Experiment with using dicts instead.
    DDmakesd = dict(zip(hrcdf['board_record_string'],hrcdf['DDmakes']))
    Pard = dict(zip(hrcdf['board_record_string'],hrcdf['Par']))
    #display(DDmakesd)
    #stdhrs = pd.merge(stdhrs,hrcdf[['EventBoard','DDmakes','Par']],on='EventBoard',how='left')
    #display(hrcdf[hrcdf['EventBoard'].isin(stdhrs['EventBoard'])]
    #display(len(stdhrs),stdhrs)
    # takes 1m
    DDmakesClean = stdhrs.apply(lambda r: DDmakesd[r['board_record_string']] if r['board_record_string'] in DDmakesd else None,axis='columns')
    b = DDmakesClean.notna() & stdhrs['DDmakes'].ne(DDmakesClean)
    assert all(~b)
    # takes 1m
    # verify that dirty Pars are still dirty
    ParClean = stdhrs.apply(lambda r: Pard[r['board_record_string']] if r['board_record_string'] in Pard else None,axis='columns')
    b = ParClean.notna() & stdhrs['Par'].ne(ParClean)
    display(stdhrs[b])
    # assign clean Pars. verify that all Pars are now cleaned.
    stdhrs.loc[b,'Par'] = ParClean
    b = ParClean.notna() & stdhrs['Par'].ne(ParClean)
    assert all(~b)
else:
    ParClean = pd.Series()

3788821

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,DDmakes,brs_dup
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))","(200, [(4, S, *, EW, -1)])","((10, 4, 10, 4, 8), (2, 9, 3, 9, 2), (10, 4, 10, 4, 8), (2, 9, 3, 9, 2))",False
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))","(90, [(1, D, , NS, 1), (1, C, , NS, 1), (1, N, , NS, 0)])","((8, 8, 7, 7, 7), (4, 5, 6, 6, 6), (8, 8, 7, 7, 7), (4, 5, 6, 6, 6))",False
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJTD65CAKTSK984H5DT8743C987SA52HAQ9832DKJC42,"((T, 764, AQ92, QJ653), (K984, 5, T8743, 987), (A52, AQ9832, KJ, 42), (QJ763, KJT, 65, AKT))","(140, [(3, H, , NS, 0)])","((9, 7, 9, 4, 6), (4, 6, 3, 8, 6), (9, 7, 9, 5, 6), (4, 6, 3, 8, 6))",False
3,"(acbl, 2620420)",77908,100961,28,200217A_28,W,N_S,STHQJ9632DJ9CJ932SKQH8DQT86542CT76SJ76HATDAK3CAKQ84SA985432HK754D7C5,"((T, QJ9632, J9, J932), (J76, AT, AK3, AKQ84), (A985432, K754, 7, 5), (KQ, 8, QT86542, T76))","(-460, [(5, N, , EW, 0)])","((1, 0, 10, 8, 1), (11, 11, 3, 5, 11), (0, 0, 10, 8, 0), (11, 11, 3, 5, 11))",False
4,"(acbl, 2620399)",77908,100961,07,200217A_07,S,Both,S2HQT762D92CAKT92SAKJ7HA8DKJ4C8643SQT854HK43DA765CQS963HJ95DQT83CJ75,"((2, QT762, 92, AKT92), (QT854, K43, A765, Q), (963, J95, QT83, J75), (AKJ7, A8, KJ4, 8643))","(-1430, [(6, S, , EW, 0)])","((6, 2, 6, 1, 3), (7, 11, 6, 12, 8), (6, 2, 6, 1, 3), (7, 11, 6, 12, 8))",False


,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,DDmakes,brs_dup
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))","(100, [(4, S, *, EW, -1)])","((10, 4, 10, 4, 8), (2, 9, 3, 9, 2), (10, 4, 10, 4, 8), (2, 9, 3, 9, 2))",False
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJTD65CAKTSK984H5DT8743C987SA52HAQ9832DKJC42,"((T, 764, AQ92, QJ653), (K984, 5, T8743, 987), (A52, AQ9832, KJ, 42), (QJ763, KJT, 65, AKT))","(100, [(3, S, *, EW, -1)])","((9, 7, 9, 4, 6), (4, 6, 3, 8, 6), (9, 7, 9, 5, 6), (4, 6, 3, 8, 6))",False
3,"(acbl, 2620420)",77908,100961,28,200217A_28,W,N_S,STHQJ9632DJ9CJ932SKQH8DQT86542CT76SJ76HATDAK3CAKQ84SA985432HK754D7C5,"((T, QJ9632, J9, J932), (J76, AT, AK3, AKQ84), (A985432, K754, 7, 5), (KQ, 8, QT86542, T76))","(-300, [(6, H, *, NS, -2)])","((1, 0, 10, 8, 1), (11, 11, 3, 5, 11), (0, 0, 10, 8, 0), (11, 11, 3, 5, 11))",False
13,"(acbl, 2620404)",77908,100961,12,200217A_12,W,N_S,SAJ63HJ8DAK2CQ765SK54H43DQJ97CAT32S87HK9765D84CKJ94SQT92HAQT2DT653C8,"((AJ63, J8, AK2, Q765), (87, K9765, 84, KJ94), (QT92, AQT2, T653, 8), (K54, 43, QJ97, AT32))","(450, [(4, S, , NS, 1)])","((7, 10, 8, 11, 9), (6, 3, 5, 2, 4), (7, 10, 8, 11, 9), (6, 3, 5, 2, 4))",False
21,"(acbl, 2620395)",77908,100961,03,200217A_03,S,E_W,SK75HK3DKQT852C52SJT63HAT2DJ64CKQTS98HJ9875DA7CA964SAQ42HQ64D93CJ873,"((K75, K3, KQT852, 52), (98, J9875, A7, A964), (AQ42, Q64, 93, J873), (JT63, AT2, J64, KQT))","(100, [(3, H, *, EW, -1)])","((5, 9, 4, 7, 5), (7, 4, 8, 6, 5), (5, 9, 4, 7, 5), (7, 4, 8, 6, 5))",False
36,"(acbl, 2915193)",86634,114736,06,200302A_06,E,E_W,ST62H984DQ6542CKJSQ4HJ765DKJTCQ853SAKJ83H3DA83CA942S975HAKQT2D97CT76,"((T62, 984, Q6542, KJ), (AKJ83, 3, A83, A942), (975, AKQT2, 97, T76), (Q4, J765, KJT, Q853))","(-450, [(4, S, , EW, 1)])","((2, 4, 6, 2, 4), (11, 8, 7, 11, 9), (2, 4, 6, 2, 4), (11, 8, 7, 11, 9))",False
41,"(acbl, 2915212)",86634,114736,25,200302A_25,N,E_W,S643H9843D975CK65SKQ7HQ5DAJ42CAQ93SAJT95HAJ6DKT6C87S82HKT72DQ83CJT42,"((643, 9843, 975, K65), (AJT95, AJ6, KT6, 87), (82, KT72, Q83, JT42), (KQ7, Q5, AJ42, AQ93))","(-990, [(6, N, , EW, 0)])","((2, 1, 3, 1, 1), (11, 12, 9, 12, 12), (2, 1, 3, 1, 1), (10, 12, 9, 12, 12))",False
43,"(acbl, 2915192)",86634,114736,05,200302A_05,N,N_S,S742HKQ743DKQ5C72SA65HT85DJ9CQJT98SKT9HJ2DAT87632C3SQJ83HA96D4CAK654,"((742, KQ743, KQ5, 72), (KT9, J2, AT87632, 3), (QJ83, A96, 4, AK654), (A65, T85, J9, QJT98))","(420, [(4, H, , NS, 0)])","((7, 5, 10, 9, 8), (6, 8, 3, 4, 5), (7, 5, 10, 9, 8), (6, 8, 3, 4, 5))",False
44,"(acbl, 2915199)",86634,114736,12,200302A_12,W,N_S,SAQHKQT74DQ3CJ432SJ765H8DJT854CQT9SK983H952D976CAK7ST42HAJ63DAK2C865,"((AQ, KQT74, Q3, J432), (K983, 952, 976, AK7), (T42, AJ63, AK2, 865), (J765, 8, JT854, QT9))","(420, [(4, H, , NS, 0)])","((9, 6, 10, 6, 9), (4, 7, 3, 7, 4), (8, 6, 10, 6, 9), (4, 7, 3, 7, 4))",False
46,"(acbl, 2915205)",86634,114736,18,200302A_18,E,N_S,SKQT5HAQ4DAKQ532CSJ9863HT63D84CJ87S2HK9875D9CAKT962SA74HJ2DJT76CQ543,"((KQT5, AQ4, AKQ532, ), (2, K9875, 9, AKT962), (A74, J2, JT76, Q543), (J9863, T63, 84, J87))","(1440, [(7, D, , NS, 0)])","((6, 13, 7, 10, 11), (7, 0, 6, 3, 2), (5, 13, 5, 9, 11), (7, 0, 6, 3, 2))",False


In [12]:
# valdiate calculated dd result by comparing against known (assumed) correct dd. Similar to functions.CompareTable().
def CompareDDTables(DDtable1, DDtable2):
    for suit in range(dds.DDS_STRAINS):
        for pl in range(4):
            if DDtable1[suit][pl] != DDtable2[4 * suit + pl]:
                return False
    return True

In [13]:
# valdiate calculated par result by comparing against known (assumed) correct par. Similar to functions.ComparePar().
def ComparePars(par1, par2):
    if par1[0] != par2[0]:
        return False
    if len(par1[1]) != len(par2[1]):
        return False
    for p1,p2 in zip(sorted(par1[1],key=lambda k: k[1]),sorted(par2[1],key=lambda k: k[1])):
        if p1 != p2:
            if p1[1] != p2[1] or p1[2] != p2[2] or p1[3] != p2[3]:
                return False # suit/double/direction difference
            if p1[0]+p1[4] != p2[0]+p2[4]:
                return False # only needs to have trick count same. ignore if levels are same (min level+overs vs max level+0)
    return True

In [14]:
# undertricks: 0 = make; 1-13 = sacrifice
# overTricks: 0-3; e.g. 1 for 4S + 1
# level: 1-7
denom = 'NSHDC'
seats = ['N','E','S','W','NS','EW']

dealer_d = {'N':0, 'E':1, 'S':2, 'W':3}
vul_d = {'None':0, 'Both':1, 'N_S':2, 'E_W':3} # dds vul encoding is weird

In [15]:
# perform initialization for calculation of double dummy (all positions and all strains) and Pars.

import dds
import ctypes
import functions

starting_handno = 0 # for restarting at a particular hando (0-len(stdhrs)). normally 0.

DDdealsPBN = dds.ddTableDealsPBN()
tableRes = dds.ddTablesRes()
pres = dds.allParResults()
presmaster = dds.parResultsMaster()

mode = 0
tFilter = ctypes.c_int * dds.DDS_STRAINS
trumpFilter = tFilter(0, 0, 0, 0, 0)
line = ctypes.create_string_buffer(80)

dds.SetMaxThreads(0)
max_tables = dds.MAXNOOFTABLES
df = stdhrs[ParClean.isna()]
nhands = len(df)
checkpoint_after_every = 1000

Loaded lib <CDLL 'c:\sw\bridge\dds\dds-291\vs2022\x64\Release\dds.dll', handle 7ffd23da0000 at 0x239338d30d0>


In [16]:
# takes 1h per 100k hands.
# Perform dd and par calcs. Restartable.
# todo: implement direction in pbn. always using 'N:' for now.
# todo: some pars in acbl_hand_records.pkl are incorrect due to ignoring of vul. Need to correct pkl in previous step.

for n,grp_start in enumerate(range(starting_handno,nhands,max_tables)):
    n += 1
    grp_count = min(nhands-grp_start,max_tables)
    DDdealsPBN.noOfTables = grp_count
    print(f'Processing group:{n} hands:{grp_start} to {grp_start+grp_count} of {nhands}')
    DDtables = []
    for handno in range(grp_count):
        r = df.iloc[grp_start+handno]
        nsew_tuple = r['Hands']
        pbn = 'N:'+' '.join('.'.join([suit for suit in suits]) for suits in nsew_tuple)
        assert len(pbn) == 1+1+13*4+3*4+3, len(pbn) # 69=='N'+':'+(13 cards per hand)+(3 '.' per suit per hand)+3 spaces
        pbn_encoded = pbn.encode()
        #print(len(pbn),pbn,pbn_encoded)
        DDdealsPBN.deals[handno].cards = pbn_encoded
        nsew_tuple = r['DDmakes']
        DDtable = [nsew_tuple[nesw][cdhsn] for cdhsn in [3,2,1,0,4] for nesw in range(4)] # convert indexing from mlb to dds
        assert len(DDtable) == 5*4, DDtable
        DDtables.append(DDtable)

    # CalcAllTablesPBN will do multi-threading
    res = dds.CalcAllTablesPBN(ctypes.pointer(DDdealsPBN), mode, trumpFilter, ctypes.pointer(tableRes), ctypes.pointer(pres))

    if res != dds.RETURN_NO_FAULT:
        dds.ErrorMessage(res, line)
        print(f'CalcAllTablesPBN: DDS error: {line.value.decode("utf-8")}')
        assert False, grp_start
    
    for handno in range(0, grp_count):
        r = df.iloc[grp_start+handno]
        #display(r)
        ref = r['ref'][1]
        dealer = r['Dealer']
        assert dealer in 'NESW'
        vul = r['Vul']
        assert vul in ['None','N_S','E_W','Both'], r
        par = r['Par']
        assert type(par) is tuple, r
        assert len(par) == 2, r
        assert type(par[0]) is int, r
        assert type(par[1]) is list, r
        assert len(par[1]) > 0, r
        par_result = ctypes.pointer(presmaster)
        dd_result = ctypes.pointer(tableRes.results[handno])

        # Par calculations are not multi-threading
        res = dds.DealerParBin(dd_result, par_result, dealer_d[dealer], vul_d[vul])
        if res != dds.RETURN_NO_FAULT:
            dds.ErrorMessage(res, line)
            print(f'DealerParBin: DDS error: {line.value.decode("utf-8")}')
            assert False, r
        
        score = par_result.contents.score
        if score == 0:
            pars = (0, [(0, '', '', '', 0)]) # par score is for everyone to pass (1 out of 100,000)
        else:
            assert par_result.contents.number > 0, r
            pars = (score,[])
            for i in range(par_result.contents.number):
                ct = par_result.contents.contracts[i]    
                #print(f'Par[{i}]: underTricks:{ct.underTricks} overTricks:{ct.overTricks} level:{ct.level} denom:{ct.denom} seats:{ct.seats}')
                assert ct.underTricks == 0 or ct.overTricks == 0
                pars[1].append((ct.level,denom[ct.denom],'*' if ct.underTricks else '',seats[ct.seats],ct.overTricks-ct.underTricks))

        dd_eq = CompareDDTables(dd_result.contents.resTable, DDtables[handno])
        par_eq = ComparePars(pars, par)
        if not dd_eq or not par_eq:
            functions.PrintPBNHand(f'Group:{n} Hand:{grp_start+handno+1}/{nhands} ACBL Ref:{ref} Vul:{vul}', DDdealsPBN.deals[handno].cards)
            functions.PrintTable(dd_result)
            if not dd_eq:
                print(f'DD mismatch: Current:{DDtables[handno]}')
                # DDtable = dd_result.contents.resTable
                #DDtable = [nsew_tuple[nesw][cdhsn] for cdhsn in [3,2,1,0,4] for nesw in range(4)] # convert indexing from mlb to dds
                # stdhrs.at[grp_start+handno,'DDmakes'] = DDtable
                assert False # temp - need to finish compatible DDtable
            if not par_eq:
                #functions.PrintDealerParBin(par_result)
                if pars[0] != par[0]: # differences in par score are usually due to vul-unknown vs vul-known.
                    print(f'Par score mismatch: Vul ignorance yielding wrong score?: Calculated:{pars[0]}, Current:{par[0]}')
                else:
                    print(f'Par contract mismatch: (Vul ignorance effecting interfering bid?): Calculated:{pars[1]}, Current:{par[1]}')
                    #assert False
                stdhrs.at[r.name,'Par'] = pars
            print()
    if n%checkpoint_after_every == 0: # save every checkpoint_after_every count (1000).
        acbl_hand_records_cleaned_filename = 'acbl_hand_records_cleaned.pkl'
        acbl_hand_records_cleaned_file = acblPath.joinpath(acbl_hand_records_cleaned_filename)
        stdhrs.to_pickle(acbl_hand_records_cleaned_file) # takes 40s
        print(f'Checkpoint at group {n}: df written to {acbl_hand_records_cleaned_file}')
        starting_handno = grp_start
starting_handno = nhands

Processing group:1 hands:0 to 40 of 217200
Processing group:2 hands:40 to 80 of 217200
Processing group:3 hands:80 to 120 of 217200
Processing group:4 hands:120 to 160 of 217200
Processing group:5 hands:160 to 200 of 217200
Processing group:6 hands:200 to 240 of 217200
Processing group:7 hands:240 to 280 of 217200
Processing group:8 hands:280 to 320 of 217200
Processing group:9 hands:320 to 360 of 217200
Processing group:10 hands:360 to 400 of 217200
Processing group:11 hands:400 to 440 of 217200
Processing group:12 hands:440 to 480 of 217200
Processing group:13 hands:480 to 520 of 217200
Processing group:14 hands:520 to 560 of 217200
Processing group:15 hands:560 to 600 of 217200
Processing group:16 hands:600 to 640 of 217200
Processing group:17 hands:640 to 680 of 217200
Processing group:18 hands:680 to 720 of 217200
Processing group:19 hands:720 to 760 of 217200
Processing group:20 hands:760 to 800 of 217200
Processing group:21 hands:800 to 840 of 217200
Processing group:22 hands:84

Processing group:169 hands:6720 to 6760 of 217200
Processing group:170 hands:6760 to 6800 of 217200
Processing group:171 hands:6800 to 6840 of 217200
Processing group:172 hands:6840 to 6880 of 217200
Processing group:173 hands:6880 to 6920 of 217200
Processing group:174 hands:6920 to 6960 of 217200
Processing group:175 hands:6960 to 7000 of 217200
Processing group:176 hands:7000 to 7040 of 217200
Processing group:177 hands:7040 to 7080 of 217200
Processing group:178 hands:7080 to 7120 of 217200
Processing group:179 hands:7120 to 7160 of 217200
Processing group:180 hands:7160 to 7200 of 217200
Processing group:181 hands:7200 to 7240 of 217200
Processing group:182 hands:7240 to 7280 of 217200
Processing group:183 hands:7280 to 7320 of 217200
Processing group:184 hands:7320 to 7360 of 217200
Processing group:185 hands:7360 to 7400 of 217200
Processing group:186 hands:7400 to 7440 of 217200
Processing group:187 hands:7440 to 7480 of 217200
Processing group:188 hands:7480 to 7520 of 217200


Processing group:330 hands:13160 to 13200 of 217200
Processing group:331 hands:13200 to 13240 of 217200
Processing group:332 hands:13240 to 13280 of 217200
Processing group:333 hands:13280 to 13320 of 217200
Processing group:334 hands:13320 to 13360 of 217200
Processing group:335 hands:13360 to 13400 of 217200
Processing group:336 hands:13400 to 13440 of 217200
Processing group:337 hands:13440 to 13480 of 217200
Processing group:338 hands:13480 to 13520 of 217200
Processing group:339 hands:13520 to 13560 of 217200
Processing group:340 hands:13560 to 13600 of 217200
Processing group:341 hands:13600 to 13640 of 217200
Processing group:342 hands:13640 to 13680 of 217200
Processing group:343 hands:13680 to 13720 of 217200
Processing group:344 hands:13720 to 13760 of 217200
Processing group:345 hands:13760 to 13800 of 217200
Processing group:346 hands:13800 to 13840 of 217200
Processing group:347 hands:13840 to 13880 of 217200
Processing group:348 hands:13880 to 13920 of 217200
Processing g

Processing group:488 hands:19480 to 19520 of 217200
Processing group:489 hands:19520 to 19560 of 217200
Processing group:490 hands:19560 to 19600 of 217200
Processing group:491 hands:19600 to 19640 of 217200
Processing group:492 hands:19640 to 19680 of 217200
Processing group:493 hands:19680 to 19720 of 217200
Processing group:494 hands:19720 to 19760 of 217200
Processing group:495 hands:19760 to 19800 of 217200
Processing group:496 hands:19800 to 19840 of 217200
Processing group:497 hands:19840 to 19880 of 217200
Processing group:498 hands:19880 to 19920 of 217200
Processing group:499 hands:19920 to 19960 of 217200
Processing group:500 hands:19960 to 20000 of 217200
Processing group:501 hands:20000 to 20040 of 217200
Processing group:502 hands:20040 to 20080 of 217200
Processing group:503 hands:20080 to 20120 of 217200
Processing group:504 hands:20120 to 20160 of 217200
Processing group:505 hands:20160 to 20200 of 217200
Processing group:506 hands:20200 to 20240 of 217200
Processing g

Processing group:646 hands:25800 to 25840 of 217200
Processing group:647 hands:25840 to 25880 of 217200
Processing group:648 hands:25880 to 25920 of 217200
Processing group:649 hands:25920 to 25960 of 217200
Processing group:650 hands:25960 to 26000 of 217200
Processing group:651 hands:26000 to 26040 of 217200
Processing group:652 hands:26040 to 26080 of 217200
Processing group:653 hands:26080 to 26120 of 217200
Processing group:654 hands:26120 to 26160 of 217200
Processing group:655 hands:26160 to 26200 of 217200
Processing group:656 hands:26200 to 26240 of 217200
Processing group:657 hands:26240 to 26280 of 217200
Processing group:658 hands:26280 to 26320 of 217200
Processing group:659 hands:26320 to 26360 of 217200
Processing group:660 hands:26360 to 26400 of 217200
Processing group:661 hands:26400 to 26440 of 217200
Processing group:662 hands:26440 to 26480 of 217200
Processing group:663 hands:26480 to 26520 of 217200
Processing group:664 hands:26520 to 26560 of 217200
Processing g

...

Processing group:4742 hands:189640 to 189680 of 217200
Processing group:4743 hands:189680 to 189720 of 217200
Processing group:4744 hands:189720 to 189760 of 217200
Processing group:4745 hands:189760 to 189800 of 217200
Processing group:4746 hands:189800 to 189840 of 217200
Processing group:4747 hands:189840 to 189880 of 217200
Processing group:4748 hands:189880 to 189920 of 217200
Processing group:4749 hands:189920 to 189960 of 217200
Processing group:4750 hands:189960 to 190000 of 217200
Processing group:4751 hands:190000 to 190040 of 217200
Processing group:4752 hands:190040 to 190080 of 217200
Processing group:4753 hands:190080 to 190120 of 217200
Processing group:4754 hands:190120 to 190160 of 217200
Processing group:4755 hands:190160 to 190200 of 217200
Processing group:4756 hands:190200 to 190240 of 217200
Processing group:4757 hands:190240 to 190280 of 217200
Processing group:4758 hands:190280 to 190320 of 217200
Processing group:4759 hands:190320 to 190360 of 217200
Processing

Processing group:4891 hands:195600 to 195640 of 217200
Processing group:4892 hands:195640 to 195680 of 217200
Processing group:4893 hands:195680 to 195720 of 217200
Processing group:4894 hands:195720 to 195760 of 217200
Processing group:4895 hands:195760 to 195800 of 217200
Processing group:4896 hands:195800 to 195840 of 217200
Processing group:4897 hands:195840 to 195880 of 217200
Processing group:4898 hands:195880 to 195920 of 217200
Processing group:4899 hands:195920 to 195960 of 217200
Processing group:4900 hands:195960 to 196000 of 217200
Processing group:4901 hands:196000 to 196040 of 217200
Processing group:4902 hands:196040 to 196080 of 217200
Processing group:4903 hands:196080 to 196120 of 217200
Processing group:4904 hands:196120 to 196160 of 217200
Processing group:4905 hands:196160 to 196200 of 217200
Processing group:4906 hands:196200 to 196240 of 217200
Processing group:4907 hands:196240 to 196280 of 217200
Processing group:4908 hands:196280 to 196320 of 217200
Processing

Processing group:5039 hands:201520 to 201560 of 217200
Processing group:5040 hands:201560 to 201600 of 217200
Processing group:5041 hands:201600 to 201640 of 217200
Processing group:5042 hands:201640 to 201680 of 217200
Processing group:5043 hands:201680 to 201720 of 217200
Processing group:5044 hands:201720 to 201760 of 217200
Processing group:5045 hands:201760 to 201800 of 217200
Processing group:5046 hands:201800 to 201840 of 217200
Processing group:5047 hands:201840 to 201880 of 217200
Processing group:5048 hands:201880 to 201920 of 217200
Processing group:5049 hands:201920 to 201960 of 217200
Processing group:5050 hands:201960 to 202000 of 217200
Processing group:5051 hands:202000 to 202040 of 217200
Processing group:5052 hands:202040 to 202080 of 217200
Processing group:5053 hands:202080 to 202120 of 217200
Processing group:5054 hands:202120 to 202160 of 217200
Processing group:5055 hands:202160 to 202200 of 217200
Processing group:5056 hands:202200 to 202240 of 217200
Processing

Processing group:5188 hands:207480 to 207520 of 217200
Processing group:5189 hands:207520 to 207560 of 217200
Processing group:5190 hands:207560 to 207600 of 217200
Processing group:5191 hands:207600 to 207640 of 217200
Processing group:5192 hands:207640 to 207680 of 217200
Processing group:5193 hands:207680 to 207720 of 217200
Processing group:5194 hands:207720 to 207760 of 217200
Processing group:5195 hands:207760 to 207800 of 217200
Processing group:5196 hands:207800 to 207840 of 217200
Processing group:5197 hands:207840 to 207880 of 217200
Processing group:5198 hands:207880 to 207920 of 217200
Processing group:5199 hands:207920 to 207960 of 217200
Processing group:5200 hands:207960 to 208000 of 217200
Processing group:5201 hands:208000 to 208040 of 217200
Processing group:5202 hands:208040 to 208080 of 217200
Processing group:5203 hands:208080 to 208120 of 217200
Processing group:5204 hands:208120 to 208160 of 217200
Processing group:5205 hands:208160 to 208200 of 217200
Processing

Processing group:5337 hands:213440 to 213480 of 217200
Processing group:5338 hands:213480 to 213520 of 217200
Processing group:5339 hands:213520 to 213560 of 217200
Processing group:5340 hands:213560 to 213600 of 217200
Processing group:5341 hands:213600 to 213640 of 217200
Processing group:5342 hands:213640 to 213680 of 217200
Processing group:5343 hands:213680 to 213720 of 217200
Processing group:5344 hands:213720 to 213760 of 217200
Processing group:5345 hands:213760 to 213800 of 217200
Processing group:5346 hands:213800 to 213840 of 217200
Processing group:5347 hands:213840 to 213880 of 217200
Processing group:5348 hands:213880 to 213920 of 217200
Processing group:5349 hands:213920 to 213960 of 217200
Processing group:5350 hands:213960 to 214000 of 217200
Processing group:5351 hands:214000 to 214040 of 217200
Processing group:5352 hands:214040 to 214080 of 217200
Processing group:5353 hands:214080 to 214120 of 217200
Processing group:5354 hands:214120 to 214160 of 217200
Processing

In [17]:
# takes 40s
acbl_hand_records_cleaned_filename = 'acbl_hand_records_cleaned.pkl'
acbl_hand_records_cleaned_file = acblPath.joinpath(acbl_hand_records_cleaned_filename)
stdhrs.to_pickle(acbl_hand_records_cleaned_file)